# 缩减单元格

In [1]:
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import numpy as np

In [2]:
# 经纬度计算距离
def geoDistance(lat1, lon1, lat2, lon2):
    # 将十进制度数转化为弧度
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # haversine公式
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r


In [12]:
# 时间戳测试
t = 1452760800
print(t)
print(str(pd.to_datetime(t, unit='s')).split(' ')[0])

1452760800
2016-01-14


In [3]:
# 字典测试
airport = pd.read_csv('./dataset/airport.csv', encoding= 'gbk')
# 读取airport.csv文件为字典
airport_dict = {}
for i in range(len(airport)):
    airport_dict[airport['机场编码'][i]] = (airport['经度'][i], airport['纬度'][i])
print(airport_dict)

{'JDZ': (117.18366, 29.27324), 'PVG': (121.48941, 31.40527), 'TGO': (122.25615, 43.62244), 'XMN': (118.03394, 24.48405), 'URC': (88.31104, 43.36378), 'TYN': (112.48699, 37.94036), 'SZX': (113.88308, 22.55329), 'SYX': (109.7525, 18.40005), 'TAO': (120.39629, 36.30744), 'TNA': (116.75199, 36.55358), 'SJW': (114.53952, 38.03647), 'SHE': (123.46987, 41.80515), 'SHA': (121.48941, 31.40527), 'PEK': (116.23128, 40.22077), 'NKG': (118.8921, 31.32751), 'KWL': (110.30188, 25.31402), 'KMG': (102.82147, 24.88554), 'HRB': (126.95717, 45.54774), 'HET': (111.62299, 40.80772), 'HGH': (120.21201, 30.2084), 'FOC': (119.27345, 26.04769), 'CTU': (104.10194, 30.65984), 'CKG': (106.54041, 29.40268), 'HFE': (117.30794, 31.79322), 'CGO': (113.6401, 34.72468), 'TLQ': (89.18596, 42.94244), 'WUH': (114.02919, 30.58203)}


In [15]:
# 改为utf-8编码 读取dataset/flight.csv文件
origin_dataset = pd.read_csv('./dataset/flights.csv', encoding='gbk')
print('成功读取flights.csv文件')
# 删除ORIGIN_AIRPORT行中没有dataset/airports.csv中的IATA_CODE的行
airport = pd.read_csv('./dataset/airport.csv', encoding= 'gbk')
print('成功读取airport.csv文件')
select_airport = airport['机场编码'].values

# 将origin_dataset中的ORIGIN_AIRPORT列中的值与select_airport中的值进行比较，如果不在select_airport中，则删除该行
origin_dataset = origin_dataset[origin_dataset['出发机场'].isin(select_airport)]
print('成功删除ORIGIN_AIRPORT不在airports.csv中的行')
# 保存缩减后的数据集
origin_dataset = origin_dataset[origin_dataset['到达机场'].isin(select_airport)]

# 删除取消的航班
origin_dataset = origin_dataset[origin_dataset['航班是否取消'] != '取消']


origin_dataset['计划出发日期'] = str(0)
origin_dataset['计划到达日期'] = str(0)
origin_dataset['实际出发日期'] = str(0)
origin_dataset['实际到达日期'] = str(0)
origin_dataset['出发延迟'] = 0
origin_dataset['到达延迟'] = 0
origin_dataset['出发经度'] = 0
origin_dataset['出发纬度'] = 0
origin_dataset['到达经度'] = 0
origin_dataset['到达纬度'] = 0
origin_dataset['距离'] = 0
# 时间码转换为时间格式
for i in range(len(origin_dataset)):
    origin_dataset['计划出发日期'].values[i] = str(pd.to_datetime(origin_dataset['计划起飞时间'].values[i],unit='s'))
    # print(origin_dataset['计划出发日期'].values[i])
    origin_dataset['计划到达日期'].values[i] = str(pd.to_datetime(origin_dataset['计划到达时间'].values[i],unit='s'))
    # print(origin_dataset['计划到达日期'].values[i])
    origin_dataset['实际出发日期'].values[i] = str(pd.to_datetime(origin_dataset['实际起飞时间'].values[i],unit='s'))
    # print(origin_dataset['实际出发日期'].values[i])
    origin_dataset['实际到达日期'].values[i] = str(pd.to_datetime(origin_dataset['实际到达时间'].values[i],unit='s'))
    # print(origin_dataset['实际到达日期'].values[i])
    if (i%100000 == 1):
        print(i/len(origin_dataset)*100,'%')

origin_dataset.to_csv('./dataset/flight_reduced.csv', index=False)

'''
# 创建新的列
# 起飞延误时间和到达延误时间

    origin_dataset['出发延迟'] = (pd.to_datetime(origin_dataset['实际出发']) -  pd.to_datetime(origin_dataset['计划出发'])) / np.timedelta64(1, 'm')
    origin_dataset['到达延迟'] = (pd.to_datetime(origin_dataset['实际到达']) -  pd.to_datetime(origin_dataset['计划到达'])) / np.timedelta64(1, 'm')
    

# 通过airport_dict创建出发经纬度与到达经纬度
# 创建新的列

    origin_dataset['出发经度'].values[i] = airport_dict[origin_dataset['出发机场'].values[i]][0]
    origin_dataset['出发纬度'].values[i] = airport_dict[origin_dataset['出发机场'].values[i]][1]
    origin_dataset['到达经度'].values[i] = airport_dict[origin_dataset['到达机场'].values[i]][0]
    origin_dataset['到达纬度'].values[i] = airport_dict[origin_dataset['到达机场'].values[i]][1]

# 通过经纬度计算距离

    origin_dataset['距离'].values[i] = geoDistance(origin_dataset['出发经度'].values[i], origin_dataset['出发纬度'].values[i], origin_dataset['到达经度'].values[i], origin_dataset['到达纬度'].values[i])
    
# origin_dataset['距离'] = geoDistance(airport[airport['机场编码'].values == origin_dataset['出发机场'].values]['经度'], airport[airport['机场编码'].values == origin_dataset['出发机场'].values]['纬度'], airport[airport['机场编码'].values == origin_dataset['到达机场'].values]['经度'], airport[airport['机场编码'].values == origin_dataset['到达机场'].values]['纬度'])
'''



成功读取flights.csv文件
成功读取airport.csv文件
成功删除ORIGIN_AIRPORT不在airports.csv中的行
4.295495743163719e-05 %
4.29553869812115 %
8.591034441284869 %
12.886530184448588 %
17.182025927612305 %
21.477521670776024 %
25.77301741393974 %
30.06851315710346 %
34.36400890026718 %
38.6595046434309 %
42.95500038659462 %
47.25049612975833 %
51.54599187292206 %
55.84148761608577 %
60.1369833592495 %
64.43247910241321 %
68.72797484557694 %
73.02347058874065 %
77.31896633190436 %
81.61446207506809 %
85.9099578182318 %
90.20545356139552 %
94.50094930455923 %
98.79644504772295 %


"\n# 创建新的列\n# 起飞延误时间和到达延误时间\n\n    origin_dataset['出发延迟'] = (pd.to_datetime(origin_dataset['实际出发']) -  pd.to_datetime(origin_dataset['计划出发'])) / np.timedelta64(1, 'm')\n    origin_dataset['到达延迟'] = (pd.to_datetime(origin_dataset['实际到达']) -  pd.to_datetime(origin_dataset['计划到达'])) / np.timedelta64(1, 'm')\n    \n\n# 通过airport_dict创建出发经纬度与到达经纬度\n# 创建新的列\n\n    origin_dataset['出发经度'].values[i] = airport_dict[origin_dataset['出发机场'].values[i]][0]\n    origin_dataset['出发纬度'].values[i] = airport_dict[origin_dataset['出发机场'].values[i]][1]\n    origin_dataset['到达经度'].values[i] = airport_dict[origin_dataset['到达机场'].values[i]][0]\n    origin_dataset['到达纬度'].values[i] = airport_dict[origin_dataset['到达机场'].values[i]][1]\n\n# 通过经纬度计算距离\n\n    origin_dataset['距离'].values[i] = geoDistance(origin_dataset['出发经度'].values[i], origin_dataset['出发纬度'].values[i], origin_dataset['到达经度'].values[i], origin_dataset['到达纬度'].values[i])\n    \n# origin_dataset['距离'] = geoDistance(airport[airport['机场编码'].values == ori

In [71]:
# 读取./dataset/flight_reduced.csv
reduced_dataset = pd.read_csv('./dataset/flight_reduced.csv')
# 删除同一出发机场重复的计划出发的列
reduced_dataset = reduced_dataset.drop_duplicates(subset=['出发机场', '计划出发日期'], keep='first')
# 保存去重后的数据集
reduced_dataset.to_csv('./dataset/flight_reduced_unique.csv', index=False)

In [78]:
# 读取./dataset/flight_reduced_unique.csv
reduced_unique_dataset = pd.read_csv('./dataset/flight_reduced_unique.csv')
# 删除取消的航班
reduced_unique_dataset = reduced_unique_dataset[reduced_unique_dataset['是否取消'] != '取消']
# 保存去重后的数据集
reduced_unique_dataset.to_csv('./dataset/flight_reduced_unique.csv', index=False)

In [21]:
# 读取./dataset/flight_reduced.csv
reduced_unique_dataset = pd.read_csv('./dataset/flight_reduced.csv')
# 计算延迟时间
reduced_unique_dataset['出发延迟'] = (pd.to_datetime(reduced_unique_dataset['实际起飞时间'], unit='s') -  pd.to_datetime(reduced_unique_dataset['计划起飞时间'], unit='s')) / np.timedelta64(1, 'm')
reduced_unique_dataset['到达延迟'] = (pd.to_datetime(reduced_unique_dataset['实际到达时间'], unit='s') -  pd.to_datetime(reduced_unique_dataset['计划到达时间'], unit='s')) / np.timedelta64(1, 'm')
print('延迟时间计算')
# 计算出发经纬度与到达经纬度
for i in range(len(reduced_unique_dataset)):
    # 通过airport_dict创建出发经纬度与到达经纬度
    
    reduced_unique_dataset['出发经度'].values[i] = airport_dict[reduced_unique_dataset['出发机场'].values[i]][0]
    reduced_unique_dataset['出发纬度'].values[i] = airport_dict[reduced_unique_dataset['出发机场'].values[i]][1]
    reduced_unique_dataset['到达经度'].values[i] = airport_dict[reduced_unique_dataset['到达机场'].values[i]][0]
    reduced_unique_dataset['到达纬度'].values[i] = airport_dict[reduced_unique_dataset['到达机场'].values[i]][1]
    # 通过经纬度计算距离
    
    reduced_unique_dataset['距离'].values[i] = geoDistance(reduced_unique_dataset['出发经度'].values[i], reduced_unique_dataset['出发纬度'].values[i], reduced_unique_dataset['到达经度'].values[i], reduced_unique_dataset['到达纬度'].values[i])
    if(i % 100000) == 0:
        print(i/len(reduced_unique_dataset)*100, '%')
# 保存计算后的数据集
reduced_unique_dataset.to_csv('./dataset/flight_reduced_calculated.csv', index=False)

延迟时间计算
0.0 %
4.295495743163719 %
8.590991486327438 %
12.886487229491156 %
17.181982972654875 %
21.47747871581859 %
25.77297445898231 %
30.06847020214603 %
34.36396594530975 %
38.65946168847347 %
42.95495743163718 %
47.2504531748009 %
51.54594891796462 %
55.841444661128335 %
60.13694040429206 %
64.43243614745577 %
68.7279318906195 %
73.02342763378321 %
77.31892337694694 %
81.61441912011065 %
85.90991486327437 %
90.20541060643808 %
94.5009063496018 %
98.79640209276552 %


# 删除多余的列

In [22]:
# 读取./dataset/flight_reduced_calculated.csv
reduced_calculated_dataset = pd.read_csv('./dataset/flight_reduced_calculated.csv')
# 删除多余的列
reduced_calculated_dataset = reduced_calculated_dataset.drop(['计划起飞时间', '计划到达时间', '实际起飞时间', '实际到达时间', '航班是否取消', '实际到达时间', '出发经度', '出发纬度', '到达经度', '到达纬度'], axis=1)

# 保存计算后的数据集
reduced_calculated_dataset.to_csv('./dataset/flight_reduced_calculated.csv', index=False)

In [23]:
# 读取./dataset/flight_reduced_calculated.csv
reduced_calculated_dataset = pd.read_csv('./dataset/flight_final.csv')
# 删除多余的列
reduced_calculated_dataset = reduced_calculated_dataset.drop(['飞机编号'], axis=1)

# 保存计算后的数据集
reduced_calculated_dataset.to_csv('./dataset/flight_final.csv', index=False)

In [25]:
# 读取./dataset/flight_final.csv
dataset = pd.read_csv('./dataset/flight_final.csv')

# 删除多余的列
dataset = dataset.drop(['计划到达日期','实际出发日期','实际到达日期','到达延迟'], axis=1)

# 保存
dataset.to_csv('./dataset/flight_final.csv', index=False)

In [3]:
# 读取./dataset/flight_final.csv
dataset = pd.read_csv('./dataset/flight_final.csv')

dataset['年'] = str(0)
dataset['月'] = str(0)
dataset['日'] = str(0)
dataset['时间'] = str(0)

# 将计划出发日期转换为年,月,日,起飞时间的列
for i in range(len(dataset)):
    dataset['年'].values[i] = dataset['计划出发日期'].values[i][0:4]
    dataset['月'].values[i] = dataset['计划出发日期'].values[i][5:7]
    dataset['日'].values[i] = dataset['计划出发日期'].values[i][8:10]
    dataset['时间'].values[i] = dataset['计划出发日期'].values[i][11:19]
    # print(dataset['年'].values[i], dataset['月'].values[i], dataset['日'].values[i], dataset['时间'].values[i])
    if(i % 100000) == 0:
        print(i/len(dataset)*100, '%')
# 删除计划出发日期行
dataset = dataset.drop(['计划出发日期'], axis=1)
# 保存
dataset.to_csv('./dataset/flight_final.csv', index=False)

0.0 %
4.295495743163719 %
8.590991486327438 %
12.886487229491156 %
17.181982972654875 %
21.47747871581859 %
25.77297445898231 %
30.06847020214603 %
34.36396594530975 %
38.65946168847347 %
42.95495743163718 %
47.2504531748009 %
51.54594891796462 %
55.841444661128335 %
60.13694040429206 %
64.43243614745577 %
68.7279318906195 %
73.02342763378321 %
77.31892337694694 %
81.61441912011065 %
85.90991486327437 %
90.20541060643808 %
94.5009063496018 %
98.79640209276552 %


# 出发延迟转化为分类预测对象

In [8]:
# 读取./dataset/flight_final.csv
dataset = pd.read_csv('./dataset/flight_final.csv')

# 根据出发延迟的大小,写入新的'延迟程度'列
for i in range(len(dataset)):
    if dataset['延迟程度'].values[i] == '正常':
        dataset['延迟程度'].values[i] = 0
    elif dataset['延迟程度'].values[i] == '轻度':
        dataset['延迟程度'].values[i] = 1
    elif dataset['延迟程度'].values[i] == '中度':
        dataset['延迟程度'].values[i] = 2
    elif dataset['延迟程度'].values[i] == '严重':
        dataset['延迟程度'].values[i] = 3

    if(i % 100000) == 0:
        print(i/len(dataset)*100, '%')
# 删除出发延迟行
# dataset = dataset.drop(['出发延迟'], axis=1)
# 保存
dataset.to_csv('./dataset/flight_final.csv', index=False)


0.0 %
4.310417287187155 %
8.62083457437431 %
12.931251861561465 %
17.24166914874862 %
21.552086435935777 %
25.86250372312293 %
30.172921010310088 %
34.48333829749724 %
38.7937555846844 %
43.104172871871555 %
47.414590159058704 %
51.72500744624586 %
56.03542473343302 %
60.345842020620175 %
64.65625930780733 %
68.96667659499448 %
73.27709388218165 %
77.5875111693688 %
81.89792845655595 %
86.20834574374311 %
90.51876303093026 %
94.82918031811741 %
99.13959760530457 %


In [5]:
# 读取./dataset/flight_final.csv
dataset = pd.read_csv('./dataset/flight_final.csv')

# 去除延迟程度中的0
dataset = dataset[dataset['延迟程度'] != '0']
# 保存
dataset.to_csv('./dataset/flight_final.csv', index=False)